## 이미지 전처리용 코드.
폴더 안의 이미지 목록을 불러오고, 이미지 크기를 출력함, 이미지를 일정 크기로 자르기 편하도록 간단한 기능을 제공.

In [1]:
import os
import cv2 as cv
import numpy as np
import random

In [2]:
location = os.getcwd()
print(location)

c:\Users\82103\Desktop\road


In [3]:
lo ='C:/Users/82103/Desktop/road/f'
file_list = os.listdir(lo)
file_list

['pikist (1).jpg',
 'pikist (2).jpg',
 'pikist (3).jpg',
 'pikist (4).jpg',
 'pikist.jpg']

In [4]:
#이미지 크기 측정
lo ='C:/Users/82103/Desktop/road/f/'
file_list = os.listdir(lo)
img = []
x = 0
y = 0
x_min = 9999
y_min = 9999

for i in range(len(file_list)):
    img = cv.imread(lo + file_list[i], cv.IMREAD_COLOR)
    print(img.shape)
    if x <= img.shape[1]:   
        x = img.shape[1]
        print([i], 'x=',x)

    if y <= img.shape[0]:   
        y = img.shape[0]
        print([i], 'y=',y)

    if x_min >= img.shape[1]:   
        x_min = img.shape[1]
        print([i],' 번 이미지가 가장 너비가 작음.', x_min)

    if y_min >= img.shape[0]:  
        y_min = img.shape[0]
        print([i], ' 번 이미지가 가장 높이가 작음.', y_min, )

print(x, y)
print(x_min, y_min)


(4096, 6144, 3)
[0] x= 6144
[0] y= 4096
[0]  번 이미지가 가장 너비가 작음. 6144
[0]  번 이미지가 가장 높이가 작음. 4096
(1333, 2000, 3)
[1]  번 이미지가 가장 너비가 작음. 2000
[1]  번 이미지가 가장 높이가 작음. 1333
(2448, 3264, 3)
(4000, 6016, 3)
(3168, 4224, 3)
6144 4096
2000 1333


In [5]:
#기준 크기 미만 이미지 제거 또는 제거 권고
lo ='C:/Users/82103/Desktop/road/f/'
file_list = os.listdir(lo)
img = []

for i in range(len(file_list)):
    img = cv.imread(lo + file_list[i], cv.IMREAD_COLOR)
    print(img.shape)

    if 300 >= img.shape[1]:   
        os.remove(lo + file_list[i])
        print([i],' 번 이미지 삭제 권고(기준 너비(x) 미달).', x_min)

    if 300 >= img.shape[0]:  
        os.remove(lo + file_list[i])
        print([i], ' 번 이미지삭제 권고(기준 높이(y) 미달).', y_min, )


(4096, 6144, 3)
(1333, 2000, 3)
(2448, 3264, 3)
(4000, 6016, 3)
(3168, 4224, 3)


In [6]:
def process(img_path, save_path):
    img = cv.imread(img_path, cv.IMREAD_COLOR)
    print('w=', str(img.shape[0]) +',', '  h=',img.shape[1])

    while(True):
        cv.imshow('image', img)
        key = cv.waitKey(0)
        if key == 27:
            break

    cv.destroyWindow('image')

    #return key

In [7]:
def simple_img_crop(img_path, save_path):

    mouse_is_pressing = False   #마우스 왼쪽 버튼 상태 체크를 위해 사용
    color = (255,255,255)   #도형 내부를 채울 때 사용할 색 지정시 사용
    
    #그리기 결과를 저장할 이미지
    img = cv.imread(img_path, cv.IMREAD_COLOR)

    # 가로세로 크기 중에서 더 긴 쪽을 기준으로 축소 비율을 계산
    max_size = 1000
    if img.shape[1] > img.shape[0]:
        scale = max_size / img.shape[1]
    else:
        scale = max_size / img.shape[0]

    new_width = int(img.shape[0] * scale)
    new_height = int(img.shape[1] * scale)

    img = cv.resize(img, (new_width, new_height), interpolation=cv.INTER_AREA)
    t_img = img.copy() #임시 이미지

    #마우스 이벤트 발생 시 호출될 함수 정의
    def mouse_callback(event, x, y, flags, param):
        global color, start_x, start_y, mouse_is_pressing

        #마우스 왼쪽 버튼을 누르면 발생하는 이벤트
        if event == cv.EVENT_LBUTTONDOWN:
            #랜덤으로(blue, green, red)로 사용될 색을 생성한다.
            color = (random.randrange(256), random.randrange(256), random.randrange(256))
            #마우스 왼쪽 버튼을 누른 것을 체크
            mouse_is_pressing = True
            #마우스 왼쪽 버튼을 누른 위치로 연산
            if (x - 150) < 0:
                x = 150
            if (y - 150) < 0:
                y = 150

            if (x + 150) > img.shape[1]:
                x = img.shape[1] - 150
            if (y + 150) > img.shape[0]:
                y = img.shape[0] - 150
            
            start_x = x - 150
            start_y = y - 150
            end_x = x + 150
            end_y = y + 150   

            cv.rectangle(t_img, (start_x, start_y), (end_x, end_y), color, 3)

    #윈도우 지정, 마우스 콜백함수를 지정
    cv.namedWindow('Image')
    cv.setMouseCallback('Image', mouse_callback)

    num = 0
    while(1):
        cv.imshow('Image',t_img)

        key = cv.waitKey(1)
    
        if key == 27: #창 닫기(esc)
            break
        elif key == ord('s'): #저장, 키보드 s키 누르면 현재 선택된 위치 저장 됨.
            fn = f'{save_path}_{num}.png'
            cv.imwrite(fn, img[start_y:start_y+300, start_x:start_x+300].copy())
            num = num + 1

    cv.destroyAllWindows() #창 닫기

In [8]:
lo ='C:/Users/82103/Desktop/road/f/' #자르기 전 원본 파일 경로
save_path = 'C:/Users/82103/Desktop/road/crop/' #자르면 저장할 경로
try:
    if not os.path.exists(save_path):
            os.mkdir(save_path)
except OSError:
        print("Error: Failed to create the directory.")

file_list = os.listdir(lo)
img = []

for i in range(len(file_list)):  
    ## 현재 ESC 누르면 지금 떠있는 이미지만 내려가고, 다음 이미지가 이어서 올라옴.
    ## 중간에 작업 중단 후 이어서 할 시 기록 보고 포문 범위를 지정해서 작업 수행할 것.

    print(file_list[i]) #파일이름 확인
    print(file_list[i][:-4]) #파일이름 확인(확장자명 제외)
    
    save_path = f'C:/Users/82103/Desktop/road/crop/{file_list[i][:-3]}'
    simple_img_crop(lo+ file_list[i], save_path)

pikist (1).jpg
pikist (1)
pikist (2).jpg
pikist (2)
pikist (3).jpg
pikist (3)
pikist (4).jpg
pikist (4)
pikist.jpg
pikist
